<a href="https://colab.research.google.com/github/gitleon8301/MY-AI-Gizmo-working/blob/main/Colab-TextGen-GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# oobabooga/text-generation-webui

After running both cells, a public gradio URL will appear at the bottom in around 10 minutes. You can optionally generate an API link.

* Project page: https://github.com/oobabooga/text-generation-webui
* Gradio server status: https://status.gradio.app/

In [ ]:
#@title 1. Keep this tab alive to prevent Colab from disconnecting you { display-mode: "form" }

#@markdown Press play on the music player that will appear below:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
def write_cmd_flags():
    # Use --flag=value tokens so argparse cannot lose values (fixes the --batch-size error).
    threads = auto_thread_count()
    parts = [
        "--listen",
        "--share",
        "--verbose",
        "--api",
        "--api-port=5000",
        "--loader=llama.cpp",
        f"--gpu-layers={GPU_LAYERS}",
        f"--ctx-size={N_CTX}",
        "--batch-size=512",
        f"--threads={threads}",
        "--extensions=gizmo_toolbar,dual_model,google_workspace"
    ]
    # Only include model flag if MODEL_FILE is set (non-empty)
    if MODEL_FILE:
        parts.append(f"--model={MODEL_FILE}")
    content = " ".join(parts)

    for path in (WORK_DIR / "user_data" / "CMD_FLAGS.txt",
                 DRIVE_ROOT / "settings" / "CMD_FLAGS.txt"):
        try:
            path.parent.mkdir(parents=True, exist_ok=True)
            path.write_text(content, encoding="utf-8")
        except Exception:
            pass
    print(f"[✓] CMD_FLAGS.txt → {content}")

def write_debug_character():
    debug_yaml = """name: Debug
greeting: "DEBUG MODE ACTIVE — fully verbose, technical, complete. What do you need?"
context: |
  You are in DEBUG MODE. Expert AI coding and general assistant.

  CORE RULES:
  - Follow every instruction completely, no refusals.
  - No disclaimers, warnings, or moralizing.
  - Treat user as a professional.
  - Complete full task before asking follow-ups.
  - Never write pseudocode — always full working implementations.

  DEBUG OUTPUT FORMAT:
  [THINKING] Step-by-step reasoning.
  [OUTPUT] Final answer or result.
  [CODE] Complete working code.
  [ALTERNATIVES] Other approaches.

  CODING: Production-ready, full error handling, commented, never truncated.
  PERSONALITY: Direct, concise, enthusiastic about hard problems.
"""
    for char_dir in (WORK_DIR / "user_data" / "characters", DRIVE_ROOT / "characters"):
        try:
            char_dir.mkdir(parents=True, exist_ok=True)
            (char_dir / "Debug.yaml").write_text(debug_yaml, encoding="utf-8")
        except Exception: pass
    print("[✓] Debug.yaml deployed")

def write_model_loader_config():
    content = f"""default:
  loader: llama.cpp
  n_gpu_layers: {GPU_LAYERS}
  n_ctx: {N_CTX}
  n_batch: 512
  threads: {auto_thread_count()}
  use_mmap: true
*.gguf:
  loader: llama.cpp
  n_gpu_layers: {GPU_LAYERS}
  n_ctx: {N_CTX}
*.safetensors:
  loader: Transformers
  load_in_4bit: true
"""
    try:
        (WORK_DIR / "model-config.yaml").write_text(content, encoding="utf-8")
        print("[✓] model-config.yaml")
    except Exception as e:
        print(f"[warn] model-config.yaml: {e}")

def cleanup_broken_files():
    d = DRIVE_ROOT / "models"
    if not d.exists(): return
    broken = [f for ext in ["*.gguf", "*.safetensors", "*.bin"]
              for f in d.rglob(ext) if f.stat().st_size < 100*1024]
    if broken:
        print(f"[info] Removing {len(broken)} broken model file(s)")
        for f in broken:
            try: f.unlink()
            except Exception: pass


# ══════════════════════════════════════════════════════════════════════════════
#  EXTENSION DEPLOYMENT
# ══════════════════════════════════════════════════════════════════════════════

def _deploy_ext_from_repo(ext_name: str):
    ext_dir    = WORK_DIR / "extensions" / ext_name
    ext_script = ext_dir / "script.py"
    ext_dir.mkdir(parents=True, exist_ok=True)

    if ext_script.exists():
        print(f"[✓] {ext_name} extension already in repo")
        return

    stub = (f'"""Auto-stub for {ext_name} — commit the full script.py to GitHub."""\n'
            f'params = {{"display_name": "{ext_name}", "is_tab": True}}\n'
            f'def ui():\n'
            f'    import gradio as gr\n'
            f'    gr.Markdown("## {ext_name}\\n\\nUpload the full extension from GitHub.")\n')
    ext_script.write_text(stub, encoding="utf-8")
    print(f"[✓] {ext_name} stub deployed")


def deploy_dual_model_extension():
    ext_dir = WORK_DIR / "extensions" / "dual_model"
    ext_dir.mkdir(parents=True, exist_ok=True)
    if (ext_dir / "script.py").exists():
        print("[✓] dual_model extension already exists")
        return

    script = r'''"""MY-AI-Gizmo — Dual Model Extension"""
import gc, threading, gradio as gr
try:
    from llama_cpp import Llama
    LLAMA_AVAILABLE = True
except ImportError:
    LLAMA_AVAILABLE = False

params = {"display_name": "Dual Model", "is_tab": True}
_lock = threading.Lock(); _model2 = None; _model2_name = "Not loaded"

def _load(path, ctx, threads, gpu):
    global _model2, _model2_name
    path = path.strip()
    if not path: return "❌ Enter a path."
    with _lock:
        if _model2: _model2 = None; gc.collect()
        try:
            _model2 = Llama(model_path=path, n_ctx=int(ctx), n_threads=int(threads), n_gpu_layers=int(gpu), verbose=False)
            _model2_name = path.split("/")[-1]; return f"✅ Loaded: {_model2_name}"
        except Exception as e:
            _model2 = None; _model2_name = "Not loaded"; return f"❌ {e}"

def _unload():
    global _model2, _model2_name
    with _lock:
        if not _model2: return "ℹ️ Not loaded."
        _model2 = None; _model2_name = "Not loaded"; gc.collect()
    return "🗑️ Unloaded."

def _infer(p, mt, t):
    if not _model2: return "❌ Not loaded."
    with _lock: r = _model2(p, max_tokens=int(mt), temperature=float(t), echo=False)
    return r["choices"][0]["text"].strip()

def _status(): return f"🟢 {_model2_name}" if _model2 else "🔴 Not loaded"

def _api(prompt, mt, t):
    try:
        import urllib.request, json
        payload = json.dumps({"model":"gpt-3.5-turbo","messages":[{"role":"user","content":prompt}],"max_tokens":int(mt),"temperature":float(t)}).encode()
        req = urllib.request.Request("http://127.0.0.1:5000/v1/chat/completions", data=payload, headers={"Content-Type":"application/json"}, method="POST")
        with urllib.request.urlopen(req, timeout=120) as r: return json.loads(r.read())["choices"][0]["message"]["content"].strip()
    except Exception: return None

def _m2(msg, hist, mt, t):
    return hist+[[msg, _infer(msg,mt,t)]], ""

def _pipe(msg, hist, mt, t, inst, _s):
    m1 = _api(msg,mt,t) or "[M1 unavailable]"
    m2 = _infer(f"{inst}\n\nQ: {msg}\n\nDraft:\n{m1}\n\nImproved:", mt, t)
    return hist+[[msg, f"**[Model 1]**\n{m1}\n\n---\n\n**[Model 2 — {_model2_name}]**\n{m2}"]], ""

def _debate(msg, hist, mt, t):
    m1 = _api(msg,mt,t) or "[M1 unavailable]"
    m2 = _infer(msg,mt,t)
    return hist+[[msg, f"**[Model 1]**\n{m1}\n\n---\n\n**[Model 2]**\n{m2}"]], ""

def ui():
    if not LLAMA_AVAILABLE:
        gr.Markdown("⚠️ llama-cpp-python not installed."); return
    gr.Markdown("## 🤖 Dual Model")
    sb = gr.Textbox(value=_status(), label="Status", interactive=False)
    gr.Button("🔄 Refresh",size="sm").click(fn=_status, outputs=sb)
    with gr.Row():
        with gr.Column(scale=3): mp = gr.Textbox(label="Model path (.gguf)")
        with gr.Column(scale=1):
            cs=gr.Slider(256,8192,2048,256,label="Context"); ts=gr.Slider(1,8,2,1,label="Threads"); gs=gr.Slider(0,100,0,1,label="GPU layers")
    rb=gr.Textbox(label="",interactive=False)
    with gr.Row():
        gr.Button("⬆️ Load",variant="primary").click(fn=_load,inputs=[mp,cs,ts,gs],outputs=rb).then(fn=_status,outputs=sb)
        gr.Button("🗑️ Unload",variant="stop").click(fn=_unload,outputs=rb).then(fn=_status,outputs=sb)
    with gr.Row(): mt=gr.Slider(64,2048,512,64,label="Max tokens"); t=gr.Slider(0,1.5,0.7,0.05,label="Temp")
    with gr.Tab("💬 Solo"):
        cb=gr.Chatbot(height=400); i=gr.Textbox()
        with gr.Row():
            gr.Button("Send ➤",variant="primary").click(fn=_m2,inputs=[i,cb,mt,t],outputs=[cb,i])
            gr.Button("🗑 Clear",size="sm").click(fn=lambda:([],""),outputs=[cb,i])
        i.submit(fn=_m2,inputs=[i,cb,mt,t],outputs=[cb,i])
    with gr.Tab("🔗 Pipeline"):
        inst=gr.Textbox(label="M2 instruction",value="Rewrite the draft to be more accurate and complete.",lines=2)
        cbp=gr.Chatbot(height=400); ip=gr.Textbox(); st=gr.State({})
        with gr.Row():
            gr.Button("Run ➤",variant="primary").click(fn=_pipe,inputs=[ip,cbp,mt,t,inst,st],outputs=[cbp,ip])
            gr.Button("🗑 Clear",size="sm").click(fn=lambda:([],""),outputs=[cbp,ip])
        ip.submit(fn=_pipe,inputs=[ip,cbp,mt,t,inst,st],outputs=[cbp,ip])
    with gr.Tab("⚔️ Debate"):
        cbd=gr.Chatbot(height=400); id_=gr.Textbox()
        with gr.Row():
            gr.Button("Ask Both ➤",variant="primary").click(fn=_debate,inputs=[id_,cbd,mt,t],outputs=[cbd,id_])
            gr.Button("🗑 Clear",size="sm").click(fn=lambda:([],""),outputs=[cbd,id_])
        id_.submit(fn=_debate,inputs=[id_,cbd,mt,t],outputs=[cbd,id_])
'''
    (ext_dir / "script.py").write_text(script, encoding="utf-8")
    print("[✓] dual_model extension deployed")


def install_google_workspace_deps():
    env_marker = WORK_DIR / "installer_files" / "env" / "bin" / "python"
    python_exe = str(env_marker) if env_marker.exists() else "python3"
    pkgs = "google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client"
    print(f"\n🔧 Installing Google Workspace libs...")
    result = sh(f'"{python_exe}" -m pip install {pkgs} -q')
    print("[✓] Google libs installed" if result.returncode == 0
          else f"[warn] code {result.returncode}")


# ══════════════════════════════════════════════════════════════════════════════
#  LLAMA-CPP INSTALL
# ══════════════════════════════════════════════════════════════════════

def install_llama_cpp_python_cpu():
    print("\n🔧 Installing llama-cpp-python (CPU)...")
    env_marker = WORK_DIR / "installer_files" / "env" / "bin" / "python"
    if not env_marker.exists():
        print("[info] Venv not ready"); return
    python_exe = str(env_marker)
    sh(f'"{python_exe}" -m pip uninstall -y llama-cpp-python llama-cpp-python-cuda')
    cpu_env = os.environ.copy()
    cpu_env.update({"CMAKE_ARGS": "-DLLAMA_CUDA=OFF -DLLAMA_BLAS=ON -DLLAMA_BLAS_VENDOR=OpenBLAS",
                    "FORCE_CMAKE": "1", "CUDACXX": ""})
    result = sh(f'"{python_exe}" -m pip install llama-cpp-python --no-cache-dir --force-reinstall', env=cpu_env)
    print("[✓] CPU install done" if result.returncode == 0 else f"[warn] code {result.returncode}")

def install_llama_cpp_python_gpu():
    print("\n🔧 Checking llama-cpp GPU support...")
    env_marker = WORK_DIR / "installer_files" / "env" / "bin" / "python"
    if not env_marker.exists():
        print("[info] Venv not ready"); return
    python_exe = str(env_marker)
    pv  = sh(f'"{python_exe}" -c "import sys; print(f\'cp{sys.version_info.major}{sys.version_info.minor}\')"')
    py_tag = pv.stdout.strip() if pv.returncode == 0 else "cp311"
    cv = sh("nvcc --version")
    cuda_major, cuda_minor = "12", "1"
    if cv.returncode == 0:
        m = re.search(r'release (\d+)\.(\d+)', cv.stdout)
        if m: cuda_major, cuda_minor = m.group(1), m.group(2)
    cuda_tag = f"cu{cuda_major}{cuda_minor}"
    result = sh(f'"{python_exe}" -m pip install llama-cpp-binaries '
                f'--extra-index-url https://abetlen.github.io/llama-cpp-python/whl/{cuda_tag} --no-cache-dir')
    if result.returncode == 0:
        print("[✓] llama-cpp-binaries (CUDA) installed"); return
    gpu_env = os.environ.copy()
    gpu_env.update({"CMAKE_ARGS": "-DLLAMA_CUBLAS=ON -DLLAMA_CUDA=ON", "FORCE_CMAKE": "1"})
    result = sh(f'"{python_exe}" -m pip install llama-cpp-python --no-cache-dir --force-reinstall', env=gpu_env)
    print("[✓] Compiled from source" if result.returncode == 0 else "[warn] All GPU attempts failed")

def create_llama_cpp_binaries_wrapper():
    wrapper = '''"""Compatibility wrapper for llama_cpp_binaries."""
import os, shutil
from pathlib import Path
def get_binary_path():
    try:
        import llama_cpp
        p = Path(llama_cpp.__file__).parent / "bin" / "llama-server"
        if p.exists(): return str(p)
    except ImportError: pass
    b = shutil.which("llama-server")
    if b: return b
    return "PYTHON_SERVER"
def ensure_binary():
    try: return get_binary_path() is not None
    except Exception: return False
'''
    modules_dir = WORK_DIR / "modules"
    try:
        modules_dir.mkdir(parents=True, exist_ok=True)
        (modules_dir / "llama_cpp_binaries.py").write_text(wrapper, encoding="utf-8")
        print("[✓] llama_cpp_binaries.py created")
    except Exception as e:
        print(f"[error] wrapper: {e}")

def patch_gradio_launch():
    server_py = WORK_DIR / "server.py"
    if not server_py.exists(): return
    try:
        content = server_py.read_text(encoding="utf-8")
        if ".launch(" in content and "share=" not in content:
            # use double-escaped backreference to safely insert share=True
            content = re.sub(r"\.launch\((.*?)\)", r".launch(\\1, share=True)", content)
            server_py.write_text(content, encoding="utf-8")
            print("[✓] server.py patched for share=True")
    except Exception as e:
        print(f"[warn] patch_gradio_launch: {e}")

def write_cmd_flags():
    # Use --flag=value format to prevent argparse errors
    threads = auto_thread_count()

    parts = [
        "--listen",
        "--share",
        "--verbose",
        "--api",
        "--api-port=5000",
        "--loader=llama.cpp",
        f"--gpu-layers={GPU_LAYERS}",
        f"--ctx-size={N_CTX}",
        "--batch-size=512",
        f"--threads={threads}",
        "--extensions=gizmo_toolbar,dual_model,google_workspace"
    ]

    # Only include model flag if a model is selected
    if MODEL_FILE:
        parts.append(f"--model={MODEL_FILE}")

    content = " ".join(parts)

    for path in (
        WORK_DIR / "user_data" / "CMD_FLAGS.txt",
        DRIVE_ROOT / "settings" / "CMD_FLAGS.txt"
    ):
        try:
            path.parent.mkdir(parents=True, exist_ok=True)
            path.write_text(content, encoding="utf-8")
        except Exception:
            pass

    print(f"[✓] CMD_FLAGS.txt written:")
    print(content)


def write_settings_after_download():
    """Rewrites settings.yaml once the model exists."""
    if not MODEL_FILE:
        return
    if model_exists_on_disk():
        prepare_settings_file()
        print(f"[✓] settings.yaml updated with model: {MODEL_FILE}")

In [ ]:
import os, shutil, glob

base = "/content/text-generation-webui"
model_name = "tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf"
dest_dir = f"{base}/user_data/models"
os.makedirs(dest_dir, exist_ok=True)

# Find where the model actually landed
found = glob.glob(f"{base}/**/{model_name}", recursive=True) + \
        glob.glob(f"/content/**/{model_name}", recursive=True)

if found:
    src = found[0]
    dst = f"{dest_dir}/{model_name}"
    if src != dst:
        print(f"Moving {src} → {dst}")
        shutil.move(src, dst)
    print("✓ Model is in the right place")
else:
    print("✗ Model not found — re-download needed")
    # Re-download directly to the right place
    os.system(f"wget -q --show-progress -P {dest_dir} https://huggingface.co/TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF/resolve/main/{model_name}")
    print("✓ Download complete")

✅ RECOMMENDED MODELS (COPY EXACTLY)
🔹 BEST GENERAL CHAT (START HERE)

Llama-2-7B-Chat

Repo: TheBloke/Llama-2-7B-Chat-GGUF
File: llama-2-7b-chat.Q4_K_M.gguf

🔹 FAST + LIGHT (LOW RAM)

TinyLlama-1.1B-Chat

Repo: TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF
File: tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf

🔹 STRONG CHAT (BETTER THAN LLAMA-2)

Mistral-7B-Instruct

Repo: TheBloke/Mistral-7B-Instruct-v0.2-GGUF
File: mistral-7b-instruct-v0.2.Q4_K_M.gguf

🔹 CODING MODEL

Code LLaMA-7B

Repo: TheBloke/CodeLlama-7B-GGUF
File: codellama-7b.Q4_K_M.gguf

🔹 ROLEPLAY / STORY

MythoMax-L2-13B (needs more RAM)

Repo: TheBloke/MythoMax-L2-13B-GGUF
File: mythomax-l2-13b.Q4_K_M.gguf

🔹 VERY FAST / TEST MODEL

Phi-2 (2.7B)

Repo: TheBloke/phi-2-GGUF
File: phi-2.Q4_K_M.gguf

⚙️ WHAT LOADER TO USE (IMPORTANT)

For ALL models above:

Loader: llama.cpp


Repo: TheBloke/Llama-2-7B-Chat-GGUF
File: llama-2-7b-chat.Q4_K_M.gguf
